In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

# Đọc dữ liệu
print("Đang đọc dữ liệu...")
csv_files = [f for f in os.listdir("weather_archive") if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError("Không tìm thấy file CSV trong thư mục weather_archive")

data_file = os.path.join("weather_archive", csv_files[0])
df = pd.read_csv(data_file)

# Kiểm tra và làm sạch dữ liệu
print("Đang xử lý dữ liệu...")
df['Date'] = pd.to_datetime(df['Date'])
df = df.dropna(subset=['Temp_Max', 'Temp_Min', 'Weather_Code'])
df['Weather_Code'] = df['Weather_Code'].astype(int)

# Định nghĩa các mã thời tiết hợp lệ và mô tả
valid_weather_codes = [0, 1, 2, 3, 45, 48, 51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]
weather_code_descriptions = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Drizzle: Light intensity",
    53: "Drizzle: Moderate intensity",
    55: "Drizzle: Dense intensity",
    56: "Freezing drizzle: Light",
    57: "Freezing drizzle: Dense",
    61: "Rain: Slight",
    63: "Rain: Moderate",
    65: "Rain: Heavy",
    66: "Freezing rain: Light",
    67: "Freezing rain: Heavy",
    71: "Snow fall: Slight",
    73: "Snow fall: Moderate",
    75: "Snow fall: Heavy",
    77: "Snow grains",
    80: "Rain showers: Slight",
    81: "Rain showers: Moderate",
    82: "Rain showers: Violent",
    85: "Snow showers: Slight",
    86: "Snow showers: Heavy",
    95: "Thunderstorm: Slight or moderate",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

# Hàm tìm mã thời tiết gần nhất
def find_nearest_weather_code(pred_code, valid_codes=valid_weather_codes):
    valid_codes = np.array(valid_codes)
    idx = (np.abs(valid_codes - pred_code)).argmin()
    return valid_codes[idx]

# Thêm các đặc trưng thời gian
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfYear'] = df['Date'].dt.dayofyear
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Mã hóa tỉnh thành
le_province = LabelEncoder()
df['Province_Code'] = le_province.fit_transform(df['Province'])

# Tạo thư mục lưu kết quả đánh giá
os.makedirs('model_evaluation', exist_ok=True)

# Lấy danh sách các tỉnh thành
provinces = df['Province'].unique()

print(f"\n{'='*50}")
print(f"ĐÁNH GIÁ MÔ HÌNH RANDOM FOREST CHO {len(provinces)} TỈNH THÀNH")
print(f"{'='*50}")

# Tạo DataFrame để lưu kết quả đánh giá
evaluation_results = []

for province_name in provinces:
    print(f"\nĐang xử lý cho tỉnh: {province_name}")
    
    # Lọc dữ liệu theo tỉnh
    province_data = df[df['Province'] == province_name].sort_values('Date')
    
    # Kiểm tra xem có đủ dữ liệu không
    if len(province_data) < 14:
        print(f"  Bỏ qua tỉnh {province_name}: không đủ dữ liệu")
        continue
    
    # Tạo các đặc trưng với độ trễ
    for lag in range(1, 8):
        province_data[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].shift(lag)
        province_data[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].shift(lag)
        province_data[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].shift(lag)
    
    # Tạo đặc trưng trung bình động
    for window in [3, 5, 7]:
        province_data[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].rolling(window=window).mean()
        province_data[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].rolling(window=window).mean()
        province_data[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].rolling(window=window).mean()
    
    # Loại bỏ các hàng có giá trị NaN
    province_data = province_data.dropna()
    
    if len(province_data) < 10:
        print(f"  Bỏ qua tỉnh {province_name}: không đủ dữ liệu sau khi tạo đặc trưng")
        continue
    
    # Chia tập dữ liệu train/test (80/20)
    train_size = int(len(province_data) * 0.8)
    train_data = province_data.iloc[:train_size]
    test_data = province_data.iloc[train_size:]
    
    # Tạo các đặc trưng cho dự đoán
    feature_cols = [col for col in train_data.columns if col.startswith(('Temp_Max_Lag', 'Temp_Min_Lag', 
                                                                        'Weather_Code_Lag', 'Temp_Max_Roll', 
                                                                        'Temp_Min_Roll', 'Weather_Code_Roll'))]
    feature_cols += ['Month', 'Day', 'DayOfYear', 'DayOfWeek']
    
    X_train = train_data[feature_cols]
    X_test = test_data[feature_cols]
    y_train_temp_max = train_data['Temp_Max']
    y_test_temp_max = test_data['Temp_Max']
    y_train_temp_min = train_data['Temp_Min']
    y_test_temp_min = test_data['Temp_Min']
    y_train_weather_code = train_data['Weather_Code']
    y_test_weather_code = test_data['Weather_Code']
    
    # Huấn luyện mô hình Random Forest
    models = {
        'temp_max': RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42, n_jobs=-1),
        'temp_min': RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42, n_jobs=-1),
        'weather_code': RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42, n_jobs=-1)
    }
    
    # Huấn luyện và đánh giá từng mô hình
    print(f"\nKết quả đánh giá cho {province_name}:")
    print("-" * 40)
    
    province_eval = {'Province': province_name}
    
    for target, model in models.items():
        if target == 'temp_max':
            y_train = y_train_temp_max
            y_test = y_test_temp_max
            target_name = "Nhiệt độ cao nhất"
        elif target == 'temp_min':
            y_train = y_train_temp_min
            y_test = y_test_temp_min
            target_name = "Nhiệt độ thấp nhất"
        else:
            y_train = y_train_weather_code
            y_test = y_test_weather_code
            target_name = "Mã thời tiết"
        
        # Huấn luyện mô hình
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Tính các metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Lưu kết quả
        province_eval[f'{target}_rmse'] = rmse
        province_eval[f'{target}_mae'] = mae
        province_eval[f'{target}_r2'] = r2
        
        # In kết quả
        print(f"{target_name}:")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MAE:  {mae:.4f}")
        print(f"  R2:   {r2:.4f}")
    
    print("-" * 40)
    
    # Thêm kết quả đánh giá của tỉnh vào danh sách
    evaluation_results.append(province_eval)

# Chuyển kết quả đánh giá thành DataFrame
evaluation_df = pd.DataFrame(evaluation_results)

# Lưu kết quả đánh giá
evaluation_df.to_csv('model_evaluation/random_forest_evaluation.csv', index=False)

print(f"\n{'='*50}")
print("ĐÃ HOÀN THÀNH ĐÁNH GIÁ MÔ HÌNH!")
print(f"Kết quả được lưu trong file: 'model_evaluation/random_forest_evaluation.csv'")
print(f"{'='*50}")

# In thống kê tổng quan
print("\nThống kê tổng quan về RMSE:")
print("-" * 40)
print(evaluation_df[['Province', 'temp_max_rmse', 'temp_min_rmse', 'weather_code_rmse']].describe())

Đang đọc dữ liệu...
Đang xử lý dữ liệu...

ĐÁNH GIÁ MÔ HÌNH RANDOM FOREST CHO 63 TỈNH THÀNH

Đang xử lý cho tỉnh: An Giang

Kết quả đánh giá cho An Giang:
----------------------------------------
Nhiệt độ cao nhất:
  RMSE: 3.7695
  MAE:  2.5988
  R2:   -0.5356
Nhiệt độ thấp nhất:
  RMSE: 0.8243
  MAE:  0.7230
  R2:   0.2041
Mã thời tiết:
  RMSE: 19.7299
  MAE:  18.8411
  R2:   -12.2225
----------------------------------------

Đang xử lý cho tỉnh: Bà Rịa - Vũng Tàu

Kết quả đánh giá cho Bà Rịa - Vũng Tàu:
----------------------------------------
Nhiệt độ cao nhất:
  RMSE: 1.5946
  MAE:  1.3127
  R2:   -1.7005
Nhiệt độ thấp nhất:
  RMSE: 1.0360
  MAE:  0.8927
  R2:   0.2232
Mã thời tiết:
  RMSE: 24.8939
  MAE:  24.0559
  R2:   -0.3715
----------------------------------------

Đang xử lý cho tỉnh: Bạc Liêu

Kết quả đánh giá cho Bạc Liêu:
----------------------------------------
Nhiệt độ cao nhất:
  RMSE: 2.7998
  MAE:  1.8592
  R2:   -1.0274
Nhiệt độ thấp nhất:
  RMSE: 0.8199
  MAE:  0.7